# DSCI 100 Group Project Report (Group 17)
## Factors of Toxicity of Mushrooms (Chosen Data Set: Mushrooms)

In [2]:
library(tidyverse)
library(tidymodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

In [24]:
# colnames <- c("id","ccf","age","sex","painloc","painexer","relrest","pncaden","cp","trestbps","htn","chol","smoke","cigs","years","fbs","dm","famhist","restecg","ekgmo","ekgday","ekgyr","dig","prop","nitr","pro","diuretic","proto","thaldur","thaltime","met","thalach","thalrest","tpeakbps","tpeakbpd","dummy","trestbpd","exang","xhypo","oldpeak","slope","rldv5","rldv5e","ca","restckm","exerckm","restef","restwm","exeref","exerwm","thal","thalsev","thalpul","earlobe","cmo","cday","cyr","num","lmt","ladprox","laddist","diag","cxmain","ramus","om1","om2","rcaprox","rcadist","lvx1","lvx2","lvx3","lvx4","lvf","cathef","junk","name")
# data <- read_delim("https://raw.githubusercontent.com/alexcw-5/DSCI-100-Group-Project/master/cleveland.csv", delim = " ", col_names=colnames)
# filtered_data <- data %>%
#                 select(thalrest, cigs, years, tpeakbps, tpeakbpd, num)
# cleaned_data <- data %>%
#                 select(-id, -ccf, -proto, -(restckm:restwm), -dummy, -name, -junk, -(lvx1:lvx4))
# cleaned_data
# nwgkjenh <- select(cleaned_data, restwm)
# nwgkjenh
data <- read_csv("https://raw.githubusercontent.com/alexcw-5/DSCI-100-Group-Project/master/mushrooms.csv")
data

Parsed with column specification:
cols(
  .default = col_character(),
  bruises = col_logical(),
  `gill-attachment` = col_logical()
)

See spec(...) for full column specifications.

Warning message:
“210 parsing failures.
 row             col           expected actual                                                                                     file
6039 gill-attachment 1/0/T/F/TRUE/FALSE      a 'https://raw.githubusercontent.com/alexcw-5/DSCI-100-Group-Project/master/mushrooms.csv'
6041 gill-attachment 1/0/T/F/TRUE/FALSE      a 'https://raw.githubusercontent.com/alexcw-5/DSCI-100-Group-Project/master/mushrooms.csv'
6376 gill-attachment 1/0/T/F/TRUE/FALSE      a 'https://raw.githubusercontent.com/alexcw-5/DSCI-100-Group-Project/master/mushrooms.csv'
6425 gill-attachment 1/0/T/F/TRUE/FALSE      a 'https://raw.githubusercontent.com/alexcw-5/DSCI-100-Group-Project/master/mushrooms.csv'
6435 gill-attachment 1/0/T/F/TRUE/FALSE      a 'https://raw.githubusercontent.com/alexcw-5/DSCI-1

class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,⋯,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<lgl>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
p,x,s,n,TRUE,p,FALSE,c,n,k,⋯,s,w,w,p,w,o,p,k,s,u
e,x,s,y,TRUE,a,FALSE,c,b,k,⋯,s,w,w,p,w,o,p,n,n,g
e,b,s,w,TRUE,l,FALSE,c,b,n,⋯,s,w,w,p,w,o,p,n,n,m
p,x,y,w,TRUE,p,FALSE,c,n,n,⋯,s,w,w,p,w,o,p,k,s,u
e,x,s,g,FALSE,n,FALSE,w,b,k,⋯,s,w,w,p,w,o,e,n,a,g
e,x,y,y,TRUE,a,FALSE,c,b,n,⋯,s,w,w,p,w,o,p,k,n,g
e,b,s,w,TRUE,a,FALSE,c,b,g,⋯,s,w,w,p,w,o,p,k,n,m
e,b,y,w,TRUE,l,FALSE,c,b,n,⋯,s,w,w,p,w,o,p,n,s,m
p,x,y,w,TRUE,p,FALSE,c,n,p,⋯,s,w,w,p,w,o,p,k,v,g


### Introduction

#### Background Information

text here

### Methodology

text here

### Results

text here

### Discussion

text here

***
## Appendix

#### Bibliography

* text here